In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import Column
from pyspark.sql.functions import upper,split,trim,ascii,substring,substring_index
from pyspark.sql.types import DecimalType,IntegerType

In [3]:
spark = SparkSession.builder.getOrCreate()
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
degree='°'

22/09/14 11:32:19 WARN Utils: Your hostname, debianwork resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
22/09/14 11:32:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/14 11:32:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
f_temps = "/home/ridvan/data/global_sea_temp/SeaTemperatures.csv" # sys.argv[1]
f_cities = "/home/ridvan/data/global_sea_temp/Cities.csv" # sys.argv[2]
f_countries = "/home/ridvan/data/global_sea_temp/Countries.csv" # sys.argv[3]

In [24]:
df_temps = spark.read \
        .option("header","true") \
        .csv(f_temps)

df_temps.show(10)

+---------+---------------+---------+--------+
|     City|       CurrTemp|WindSpeed|Humidity|
+---------+---------------+---------+--------+
|  Algiers|23.6°C / 74.5°F|   14 mph|     65%|
|   Annaba|24.9°C / 76.8°F|    3 mph|     88%|
|      Bab|23.5°C / 74.3°F|    5 mph|     52%|
|   Bejaïa|24.3°C / 75.7°F|    5 mph|     78%|
|     Beni|21.8°C / 71.2°F|    4 mph|     75%|
|    Bordj|23.5°C / 74.3°F|    8 mph|     88%|
|Boumerdas|23.4°C / 74.2°F|   13 mph|     55%|
|   Dellys|24.5°C / 76.1°F|    8 mph|     59%|
|    Jijel|24.3°C / 75.7°F|    2 mph|     83%|
|     Mers|21.8°C / 71.2°F|    6 mph|     88%|
+---------+---------------+---------+--------+
only showing top 10 rows



In [25]:
df_cities = spark.read \
        .option("header","true") \
        .csv(f_cities)

df_cities.show(10)

+-------+---------------+
|Country|           City|
+-------+---------------+
|Algeria|        Algiers|
|Algeria|         Annaba|
|Algeria|    Bab Ezzouar|
|Algeria|         Bejaïa|
|Algeria|       Beni Saf|
|Algeria|Bordj el Kiffan|
|Algeria|      Boumerdas|
|Algeria|         Dellys|
|Algeria|          Jijel|
|Algeria|  Mers el Kebir|
+-------+---------------+
only showing top 10 rows



In [43]:
df_countries = spark.read \
        .option("header","true") \
        .csv(f_countries)

df_countries = df_countries.select(
    trim(df_countries.Continent).alias('Continent'),
    trim(df_countries.Country).alias('Country')
)

df_countries.show(10)

+---------+-----------------+
|Continent|          Country|
+---------+-----------------+
|   Africa|          Algeria|
|   Africa|           Angola|
|   Africa|         Cameroon|
|   Africa|       Cape Verde|
|   Africa|            Congo|
|   Africa|         Djibouti|
|   Africa|            Egypt|
|   Africa|Equatorial Guinea|
|   Africa|          Eritrea|
|   Africa|            Gabon|
+---------+-----------------+
only showing top 10 rows



In [29]:
df_temps_clean = df_temps.select(
        df_temps.City,
        substring_index(split(df_temps.CurrTemp,'/').getItem(0),degree,1)\
                .cast(DecimalType(4,1))
                .alias('temp_c'),
        substring_index(split(df_temps.CurrTemp,'/').getItem(1),degree,1)\
                .cast(DecimalType(4,1))\
                .alias('temp_f'),
        substring_index(df_temps.WindSpeed,' mph',1)\
                .cast('integer')\
                .alias('windspeed_mph'),
        substring_index(df_temps.Humidity,'%',1)\
                .cast('integer')\
                .alias('humidity_pct')
                )

df_temps_clean.show(10)

+---------+------+------+-------------+------------+
|     City|temp_c|temp_f|windspeed_mph|humidity_pct|
+---------+------+------+-------------+------------+
|  Algiers|  23.6|  74.5|           14|          65|
|   Annaba|  24.9|  76.8|            3|          88|
|      Bab|  23.5|  74.3|            5|          52|
|   Bejaïa|  24.3|  75.7|            5|          78|
|     Beni|  21.8|  71.2|            4|          75|
|    Bordj|  23.5|  74.3|            8|          88|
|Boumerdas|  23.4|  74.2|           13|          55|
|   Dellys|  24.5|  76.1|            8|          59|
|    Jijel|  24.3|  75.7|            2|          83|
|     Mers|  21.8|  71.2|            6|          88|
+---------+------+------+-------------+------------+
only showing top 10 rows



In [45]:
df_temps2 = df_temps_clean\
    .join(df_cities, ['City'])\
    .join(df_countries,['Country'])

df_temps2.show(10) 

+-------+----------+------+------+-------------+------------+---------+
|Country|      City|temp_c|temp_f|windspeed_mph|humidity_pct|Continent|
+-------+----------+------+------+-------------+------------+---------+
|Algeria|   Algiers|  23.6|  74.5|           14|          65|   Africa|
|Algeria|    Annaba|  24.9|  76.8|            3|          88|   Africa|
|Algeria|    Bejaïa|  24.3|  75.7|            5|          78|   Africa|
|Algeria| Boumerdas|  23.4|  74.2|           13|          55|   Africa|
|Algeria|    Dellys|  24.5|  76.1|            8|          59|   Africa|
|Algeria|     Jijel|  24.3|  75.7|            2|          83|   Africa|
|Algeria|Mostaganem|  22.4|  72.3|            5|          78|   Africa|
|Algeria|      Oran|  21.8|  71.3|            8|          88|   Africa|
|Algeria|    Skikda|  24.5|  76.1|            2|          75|   Africa|
|Algeria|    Tipasa|  23.3|  73.9|            2|          62|   Africa|
+-------+----------+------+------+-------------+------------+---

In [41]:
dir(df_countries.Country == df_cities.Country)

['__add__',
 '__and__',
 '__bool__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__div__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__invert__',
 '__iter__',
 '__le__',
 '__lt__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmod__',
 '__rmul__',
 '__ror__',
 '__rpow__',
 '__rsub__',
 '__rtruediv__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__weakref__',
 '_asc_doc',
 '_asc_nulls_first_doc',
 '_asc_nulls_last_doc',
 '_bitwiseAND_doc',
 '_bitwiseOR_doc',
 '_bitwiseXOR_doc',
 '_contains_doc',
 '_desc_doc',
 '_desc_nulls_first_doc',
 '_desc_nulls_last_doc',
 '_endswith_doc',
 '_eqNullSafe_doc',
 '_ilike_doc',
 '_isNotNull_doc',
 '_is